In [ ]:
### Structure
# Extract sample data from BigQuery
# Exploring data
# Quality control and other preprocessing
# Create ML datasets
# Verify that datasets exist
# Benchmark
# Benchmark on same dataset

In [8]:
# Extract sample data from BigQuery
# Import libraries
from google.cloud import bigquery
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import shutil

In [9]:
# SQL query
sql = """
    SELECT
        pickup_datetime, 
        pickup_longitude, 
        pickup_latitude, 
        dropoff_longitude, 
        dropoff_latitude, 
        passenger_count, 
        trip_distance, 
        tolls_amount, 
        fare_amount, 
        total_amount 
    FROM `nyc-tlc.yellow.trips`
    LIMIT 10
"""

In [10]:
# Run a query
CREDS = 'arboreal-parser-228610-683598fe8b4a.json'
client = bigquery.Client.from_service_account_json(json_credentials_path=CREDS)
trips = client.query(sql).to_dataframe()
trips

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,tolls_amount,fare_amount,total_amount
0,2015-04-17 08:07:24+00:00,-73.901932,40.764160,-73.902008,40.764130,5,0.0,0.0,0.0,0.0
1,2010-02-06 20:23:08+00:00,-73.795666,40.644648,-73.980812,40.764683,2,178.7,0.0,0.0,0.0
2,2010-02-02 23:17:55+00:00,-73.924135,40.752351,-73.924135,40.752351,1,0.0,0.0,0.0,0.0
3,2013-08-08 15:21:50+00:00,-74.000860,40.762344,-74.000997,40.762137,1,0.0,0.0,0.0,0.0
4,2010-02-18 12:29:13+00:00,-73.976301,40.732448,-73.976340,40.732448,1,0.0,0.0,0.0,0.0
5,2010-03-16 17:13:04+00:00,-73.971133,40.760423,-73.786367,40.638546,1,16.1,0.0,0.0,0.0
6,2010-03-04 04:25:37+00:00,-73.776251,40.645725,-73.776251,40.645725,1,0.0,0.0,0.0,0.0
7,2013-08-31 23:59:24+00:00,-73.995712,40.723319,-74.001071,40.727537,2,4.4,0.0,0.0,0.0
8,2015-04-12 05:38:51+00:00,-73.937500,40.758331,-73.937523,40.758335,1,0.0,0.0,0.0,0.0
9,2015-05-24 23:47:47+00:00,-73.990479,40.757786,-74.023338,40.759418,2,3.0,0.0,0.0,0.3


In [ ]:
sql = """
    SELECT
        pickup_datetime,
        pickup_longitude,
        pickup_latitude,
        dropoff_longitude,
        dropoff_latitude,
        passenger_count,
        trip_distance,
        tolls_amount,
        fare_amount,
        total_amount
    FROM
        `nyc-tlc.yellow.trips`
    WHERE
        ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 100000)) = 1
"""

In [ ]:
trips = client.query(sql).to_dataframe()
trips[:10]

In [ ]:
# Exploring data
ax = sns.regplot(x="trip_distance", y="fare_amount", fit_reg=False, ci=None, truncate=True, data=trips)
ax.figure.set_size_inches(10, 8)

In [ ]:
# Modify the query to keep only trips longer than zero miles and fare amounts that are at least the minimum cab fare ($2.50)
sql = """
    SELECT
        pickup_datetime,
        pickup_longitude,
        pickup_latitude,
        dropoff_longitude,
        dropoff_latitude,
        passenger_count,
        trip_distance,
        tolls_amount,
        fare_amount,
        total_amount
    FROM
        `nyc-tlc.yellow.trips`
    WHERE
        ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 100000)) = 1
        AND trip_distance > 0 AND fare_amount >= 2.5
"""

In [ ]:
trips = client.query(sql).to_dataframe()
ax = sns.regplot(x="trip_distance", y="fare_amount", fit_reg=False, ci=None, truncate=True, data=trips)
ax.figure.set_size_inches(10 , 8)

In [ ]:
tollrides = trips[trips['tolls_amount'] > 0]
tollrides[tollrides['pickup_datetime'] == '2010-04-29 12:28:00']

In [ ]:
trips.describe()

In [ ]:
def showrides(df, numlines):
    lats = []
    lons = []
    for iter, row in df[:numlines].iterrows():
        lons.append(row['pickup_longitude'])
        lons.append(row['dropoff_longitude'])
        lons.append(None)
        lats.append(row['pickup_latitude'])
        lats.append(row['dropoff_latitude'])
        lats.append(None)
        
    sns.set_style("darkgrid")
    plt.figure(figsize=(10,8))
    plt.plot(lons, lats)
    
showrides(trips, 10)

In [ ]:
showrides(tollrides, 10)

# Quality control and other preprocessing

We need to some clean-up of the data:

1. New York city longitudes are around -74 and latitudes are around 41.
2. We shouldn't have zero passengers.
3. Clean up the total_amount column to reflect only fare_amount and tolls_amount, and then remove those two columns.
4. Before the ride starts, we'll know the pickup and dropoff locations, but not the trip distance (that depends on the route taken), so remove it from the ML dataset
5. Discard the timestamp

In [ ]:
def preprocess(trips_in):
    trips = trips_in.copy(deep=True)
    trips.fare_amount = trips.fare_amount + trips.tolls_amount
    del trips['tolls_amount']
    del trips['total_amount']
    del trips['trip_distance']
    del trips['pickup_datetime']
    qc = np.all([\
                 trips['pickup_longitude'] > -78, \
                 trips['pickup_longitude'] < -70, \
                 trips['dropoff_longitude'] > -78, \
                 trips['dropoff_longitude'] < -70, \
                 trips['pickup_latitude'] > 37, \
                 trips['pickup_latitude'] < 45, \
                 trips['dropoff_latitude'] > 37, \
                 trips['dropoff_latitude'] < 45, \
                 trips['passenger_count'] > 0,
                ], axis=0)
    return trips[qc]

tripsqc = preprocess(trips)
tripsqc.describe()

In [ ]:
# Create ML datasets
shuffled = tripsqc.sample(frac=1)
trainsize = int(len(shuffled['fare_amount']) * 0.7)
validsize = int(len(shuffled['fare_amount']) * 0.15)

df_train = shuffled.iloc[:trainsize, :]
df_valid = shuffled.iloc[trainsize:(trainsize+validsize), :]
df_test = shuffled.iloc[(trainsize+validsize):, :]

In [ ]:
df_train.describe()

In [ ]:
df_valid.describe()

In [ ]:
def to_csv(df, filename):
    outdf = df.copy(deep=False)
    outdf.loc[:, 'key'] = np.arange(0, len(outdf))  # rownumber as key
    # reorder columns so that target is first column
    cols = outdf.columns.tolist()
    cols.remove('fare_amount')
    cols.insert(0, 'fare_amount')
    print(cols)  # new order of columns
    outdf = outdf[cols]
    outdf.to_csv(filename, header=False, index_label=False, index=False)
    
to_csv(df_train, 'taxi-train.csv')
to_csv(df_valid, 'taxi-valid.csv')
to_csv(df_test, 'taxi-test.csv')

In [ ]:
# Verify that datasets exist
!ls -l *.csv

In [ ]:
!head -10 taxi-valid.csv

# Benchmark 
Before we start building complex ML models, it is a good idea to come up with a very simple model and use that as a benchmark.

My model is going to be to simply divide the mean fare_amount by the mean trip_distance to come up with a rate and use that to predict. Let's compute the RMSE of such a model.

In [ ]:
def distance_between(lat1, lon1, lat2, lon2):
    # harvesine formula to compute distance "as the crow flies". 
    # Taxis can't fly of course.
    dist = np.degrees(np.arccos(np.minimum(1,np.sin(np.radians(lat1)) * np.sin(np.radians(lat2)) + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.cos(np.radians(lon2 - lon1))))) * 60 * 1.515 * 1.609344
    return dist

def estimate_distance(df):
    return distance_between(df['pickuplat'], df['pickuplon'], df['dropofflat'], df['dropofflon'])

def compute_rmse(actual, predicted):
    return np.sqrt(np.mean((actual-predicted) ** 2))

def print_rmse(df, rate, name):
    print("{1} RMSE = {0}".format(compute_rmse(df['fare_amount'], rate*estimate_distance(df)), name))
    
FEATURES = ['pickuplon', 'pickuplat', 'dropofflon', 'dropofflat', 'passengers']
TARGET = 'fare_amount'
columns = list([TARGET])
columns.extend(FEATURES)  # in CSV, target is the first column, after the features
columns.append('key')
df_train = pd.read_csv('taxi-train.csv', header=None, names=columns)
df_valid = pd.read_csv('taxi-valid.csv', header=None, names=columns)
df_test = pd.read_csv('taxi-test.csv', header=None, names=columns)
rate = df_train['fare_amount'].mean() / estimate_distance(df_train).mean()
print("Rate = ${0}/km".format(rate))
print_rmse(df_train, rate, 'Train')
print_rmse(df_valid, rate, 'Valid')
print_rmse(df_test, rate, 'Test')

In [ ]:
# Benchmark on same dataset
def create_query(phase, EVERY_N):
    """
    pahse: 1=train 2=valid
    """
    base_query = """
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  CONCAT(CAST(pickup_datetime AS STRING), CAST(pickup_longitude AS STRING), CAST(pickup_latitude AS STRING), CAST(dropoff_latitude AS STRING), CAST(dropoff_longitude AS STRING)) AS key,
  EXTRACT(DAYOFWEEK FROM pickup_datetime)*1.0 AS dayofweek,
  EXTRACT(HOUR FROM pickup_datetime)*1.0 AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers
FROM
  `nyc-tlc.yellow.trips`
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
"""
    
    if EVERY_N == None:
        if phase < 2:
            # training
            query = "{0} AND ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 4)) < 2".format(base_query)
        else:
            query = "{0} AND ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 4)) = {1}".format(base_query, phase)
    else:
            query = "{0} AND ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), {1})) = {2}".format(base_query, EVERY_N, phase)
    
    return query

query = create_query(2, 100000)
df_valid = client.query(query).to_dataframe()
print_rmse(df_valid, 2.56, 'Final Validation Set')